In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

In [13]:
from pathlib import Path

# Path to the data directory
data_dir = Path("/content/gdrive/My Drive/newData2/")


traindf=pd.read_csv("/content/gdrive/My Drive/newData2/data.csv", dtype=str)
labels = traindf["labels"] 

# pad labels to length 16
for i in range(len(labels)):
    if(len(labels[i]) == 10):
        labels[i] = labels[i] + "      "

# Get list of all the images
images = sorted(list(map(str, list(data_dir.glob("*.png")))))

characters = set(char for label in labels for char in label )
characters.remove(' ')
print("Number of images found: ", len(images))
print("Number of labels found: ", len(labels))
print("Number of unique characters: ", len(characters))
print("Characters present: ", characters)

# Batch size for training and validation
batch_size = 32

# Desired image dimensions
img_width = 200
img_height = 50

# Maximum length of sequences in the dataset
max_length = max([len(label) for label in labels])

Number of images found:  5000
Number of labels found:  5000
Number of unique characters:  10
Characters present:  {'6', '7', '1', '8', '5', '3', '0', '4', '2', '9'}


In [4]:
# Mapping characters to integers
char_to_num = layers.StringLookup(
    vocabulary=list(characters),max_tokens = 16,pad_to_max_tokens=True, mask_token=None
)

# Mapping integers back to original characters
num_to_char = layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True
)

def split_data(images, labels, train_size=0.9, shuffle=True):
    # 1. Get the total size of the dataset
    size = len(images)
    # 2. Make an indices array and shuffle it, if required
    indices = np.arange(size)
    if shuffle:
        np.random.shuffle(indices)
    # 3. Get the size of training samples
    train_samples = int(size * train_size)
    # 4. Split data into training and validation sets
    x_train, y_train = images[indices[:train_samples]], labels[indices[:train_samples]]
    x_valid, y_valid = images[indices[train_samples:]], labels[indices[train_samples:]]
    return x_train, x_valid, y_train, y_valid


# Splitting data into training and validation sets
x_train, x_valid, y_train, y_valid = split_data(np.array(images), np.array(labels))

def encode_single_sample(img_path, label):
    # 1. Read image
    img = tf.io.read_file(img_path)
    # 2. Decode and convert to grayscale
    img = tf.io.decode_png(img, channels=1)
    # 3. Convert to float32 in [0, 1] range
    img = tf.image.convert_image_dtype(img, tf.float32)
    # 4. Resize to the desired size
    img = tf.image.resize(img, [img_height, img_width])
    # 5. Transpose the image because we want the time
    # dimension to correspond to the width of the image.
    img = tf.transpose(img, perm=[1, 0, 2])
    # 6. Map the characters in label to numbers
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))

    return {"image": img, "label": label}

In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = (
    train_dataset.map(
        encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE
    )
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

validation_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
validation_dataset = (
    validation_dataset.map(
        encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE
    )
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

In [8]:
class CTCLayer(layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # At test time, just return the computed predictions
        return y_pred


def build_model():
    # Inputs to the model
    input_img = layers.Input(
        shape=(img_width, img_height, 1), name="image", dtype="float32"
    )
    labels = layers.Input(name="label", shape=(None,), dtype="float32")

    # First conv block
    x = layers.Conv2D(
        64,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv1",
    )(input_img)
    x = layers.MaxPooling2D((2, 2), name="pool1")(x)
    x = layers.Dropout(0.2)(x)

    # Second conv block
    x = layers.Conv2D(
        128,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv2",
    )(x)
    x = layers.MaxPooling2D((2, 2), name="pool2")(x)
    x = layers.Dropout(0.2)(x)

    # We have used two max pool with pool size and strides 2.
    # Hence, downsampled feature maps are 4x smaller. 
    # The number of filters in the last layer is 128.
    new_shape = ((img_width // 4), (img_height // 4) * 128)
    x = layers.Reshape(target_shape=new_shape, name="reshape")(x)
    x = layers.Dense(64, activation="relu", name="dense1")(x)
    x = layers.Dropout(0.2)(x)

    # RNNs
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.25))(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.25))(x)

    # Output layer
    x = layers.Dense(
        len(char_to_num.get_vocabulary()) + 1, activation="softmax", name="dense2"
    )(x)

    # Add CTC layer for calculating CTC loss at each step
    output = CTCLayer(name="ctc_loss")(labels, x)

    # Define the model
    model = keras.models.Model(
        inputs=[input_img, labels], outputs=output, name="ocr_model"
    )
    # Optimizer
    opt = keras.optimizers.Adam()
    # Compile the model and return
    model.compile(optimizer=opt)
    return model

In [9]:
model = build_model()
model.summary()

Model: "ocr_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image (InputLayer)             [(None, 200, 50, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 Conv1 (Conv2D)                 (None, 200, 50, 64)  640         ['image[0][0]']                  
                                                                                                  
 pool1 (MaxPooling2D)           (None, 100, 25, 64)  0           ['Conv1[0][0]']                  
                                                                                                  
 dropout_3 (Dropout)            (None, 100, 25, 64)  0           ['pool1[0][0]']          

In [14]:
early_stopping_patience = 10
# Add early stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=early_stopping_patience, restore_best_weights=True
)

def train_and_save(epochs):
  # Train the model
  history = model.fit(
      train_dataset,
      validation_data=validation_dataset,
      epochs=epochs,
      callbacks=[early_stopping],
  )
  # save the model
  model.save("/content/gdrive/My Drive/saved_model_final")


for i in range(10):
  train_and_save(5)

Epoch 1/5
282/282 [==============================] - 55s 195ms/step - loss: 36.2727 - val_loss: 35.7176
Epoch 2/5
282/282 [==============================] - 32s 114ms/step - loss: 35.5211 - val_loss: 34.7348
Epoch 3/5
282/282 [==============================] - 33s 116ms/step - loss: 34.8430 - val_loss: 34.2079
Epoch 4/5
282/282 [==============================] - 32s 114ms/step - loss: 34.5797 - val_loss: 34.0663
Epoch 5/5
282/282 [==============================] - 32s 113ms/step - loss: 34.4033 - val_loss: 34.0325


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/saved_model_final/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/saved_model_final/assets


Epoch 1/5
282/282 [==============================] - 32s 112ms/step - loss: 34.3691 - val_loss: 33.9708
Epoch 2/5
282/282 [==============================] - 32s 113ms/step - loss: 34.2954 - val_loss: 33.9612
Epoch 3/5
282/282 [==============================] - 32s 112ms/step - loss: 34.2551 - val_loss: 34.0170
Epoch 4/5
282/282 [==============================] - 32s 112ms/step - loss: 34.2426 - val_loss: 33.8916
Epoch 5/5
282/282 [==============================] - 32s 113ms/step - loss: 34.1901 - val_loss: 33.8900


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/saved_model_final/assets


...

INFO:tensorflow:Assets written to: /content/gdrive/My Drive/saved_model_final/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/saved_model_final/assets


Epoch 1/5
282/282 [==============================] - 32s 112ms/step - loss: 32.7781 - val_loss: 32.4261
Epoch 2/5
282/282 [==============================] - 32s 112ms/step - loss: 32.6637 - val_loss: 32.4933
Epoch 3/5
282/282 [==============================] - 32s 112ms/step - loss: 32.6359 - val_loss: 32.2943
Epoch 4/5
282/282 [==============================] - 32s 112ms/step - loss: 32.5472 - val_loss: 32.2918
Epoch 5/5
282/282 [==============================] - 32s 112ms/step - loss: 32.6158 - val_loss: 32.3770


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/saved_model_final/assets


INFO:tensorflow:Assets written to: /content/gdrive/My Drive/saved_model_final/assets


In [23]:
# function to decode the output of the network
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][:, :max_length]
    # Iterate over the results and get back the text
    output_text = []
    for res in results:
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res)
    return output_text


In [24]:
def prediction(images):
    result = []
    test_dataset = tf.data.Dataset.from_tensor_slices((images, ['0' * 16] * len(images)))
    test_dataset = (
        test_dataset.map(
            encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE
        )
        .batch(batch_size)
        .prefetch(buffer_size=tf.data.AUTOTUNE)
    )

    # CHANGE
    # please put appropriate address to load the model
    prediction_model = keras.models.load_model(f"/content/gdrive/My Drive/saved_model_final")
    prediction_model = keras.models.Model(
        prediction_model.get_layer(name="image").input, prediction_model.get_layer(name="dense2").output
    )

    for batch in test_dataset.take(1):
        batch_images = batch["image"]
        batch_labels = batch["label"]

        preds = prediction_model.predict(batch_images)
        pred_texts = decode_batch_predictions(preds)

        result = result + pred_texts

    return result

In [16]:
# please make a 'Tests' directory
# then load png images inside
data_dir = Path('/content/Tests/')
images = list(map(str, list(data_dir.glob("*.png"))))

In [25]:
prediction(images)

['62383388344[UNK][UNK][UNK][UNK][UNK]']